# Libraries

In [ ]:
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

: 

# API Key Generation and API URL Declaration

In [ ]:
#API KEY
api_key = "78aac9217e5b16253e383fb61661f079"

#API URL
api_url = "https://api.the-odds-api.com/v4/sports"

#Parameters
params = {"apiKey": api_key}

#Use requests to Query API
response = requests.get(api_url, params=params)

: 

# Connect To API and Display Sports

In [ ]:
#Print API Status
if response.status_code == 200:
    sports_list = response.json()
    print("API Connected")
else:
    print("Failed to Connect:", response.status_code)
    sports_list = []

# Prints all available sports and indexes for users to choose from
print("\nAvailable Sports:")
for i, sport in enumerate(sports_list):
    print(f"[{i + 1}] {sport['title']}")

: 

# Sport Selection

In [ ]:
selected_sport = int(input("\nEnter the number of the sport you want to analyze: ")) - 1
sport_key = sports_list[selected_sport]['key']
print(f"\nYou selected: {sports_list[selected_sport]['title']}")


: 

# Region and Attributes that We are Analyzing

In [ ]:
region = "us"
markets = "spreads,h2h,totals"

: 

# Fetching Data for Spreads, Head to Head, and Totals
### These are the Only Free Metrics from our API

In [ ]:
url = f"https://api.the-odds-api.com/v4/sports/{sport_key}/odds"
params = {
    "apiKey": api_key,
    "regions": region,
    "markets": markets,
    "oddsFormat": "american"
}
response = requests.get(url, params=params)
data = response.json()


: 

### Fetching All Market Data Within Data

In [ ]:
if isinstance(data, list):
    market_types = {
        market["key"]
        for match in data
        for site in match.get("bookmakers", [])
        for market in site.get("markets", [])
    }
    
    if market_types:
        print("Market Types Found in Data")
        print(sorted(market_types))
    else:
        print("No market types found in the data.")
else:
    print("API Error or unexpected response:")
    print(data)


: 

# Raw Data

In [ ]:
print("Raw Data\n",data)

: 

# Data Cleaning

In [ ]:
df_clean = []
for match in data:
    game_time = datetime.fromisoformat(match["commence_time"].replace("Z", "+00:00"))
    home = match["home_team"]
    away = match["away_team"]
    for site in match.get("bookmakers", []):
        book = site["title"]
        for market in site.get("markets", []):
            for outcome in market["outcomes"]:
                attributes = {
                    "time": game_time,
                    "home_team": home,
                    "away_team": away,
                    "matchup": f"{away} @ {home}",
                    "market": market["key"],
                    "sportsbook": book,
                    "team": outcome["name"],
                    "odds": outcome["price"]
                }
                if "point" in outcome:
                    attributes["point"] = outcome["point"]
                df_clean.append(attributes)

: 

# Data Entry Statistics

In [ ]:
print("Loaded", len(df_clean), "entries")
final= pd.DataFrame(df_clean)
final.head()

: 

# Metric Creation


## Different Metrics We Created
###### 1. Win Score: Weights both the Spread as well as Spread Probability equally to predict a Winner
###### 2. Implied Win Probability: Which team is likely to Win According to the Book

##### Creating a Spread Score

In [ ]:
spreads = final[final["market"] == "spreads"].copy()
spreads["spread_score"] = -spreads["point"]

: 

##### Spread Odds of Converting

In [ ]:
# Step 1: Define the odds converter again (just in case it's not already defined)
def converted_odds(odds):
    return 100 / (odds + 100) if odds > 0 else -odds / (-odds + 100)

# Step 2: Prepare head-to-head (moneyline) data
h2h = final[final['market'] == 'h2h'].copy()
h2h["win_probability"] = h2h["odds"].apply(converted_odds)

# Step 3: Prepare spreads data and make sure win_score is there
spreads = final[final["market"] == "spreads"].copy()
spreads["spread_score"] = -spreads["point"]
spreads["prob"] = spreads["odds"].apply(converted_odds)
spreads["win_score"] = 0.5 * spreads["spread_score"] + 0.5 * spreads["prob"]

# Step 4: Merge h2h and spread data on team name
value_df = h2h.merge(
    spreads[["team", "win_score"]],
    on="team",
    how="left"
)

# Step 5: Calculate value score = model's win_score - book's implied win probability
value_df["value_score"] = value_df["win_score"] - value_df["win_probability"]

# Keep the highest value_score per team
best_value_bets = value_df.loc[
    value_df.groupby("team")["value_score"].idxmax()
].reset_index(drop=True)

# Optional: Filter for non-heavy favorites
best_value_bets = best_value_bets[best_value_bets["odds"] > -200]

# Sort for best bets
best_value_bets = best_value_bets.sort_values(by="value_score", ascending=False)

# Display with matchup and date info
display(best_value_bets[["team", "matchup", "date", "odds", "win_probability", "win_score", "value_score"]].head(10))


: 

In [ ]:
def converted_odds(odds):
    return 100 / (odds + 100) if odds > 0 else -odds / (-odds + 100)

spreads["prob"] = spreads["odds"].apply(converted_odds)

: 

##### Win Score: Creates a Score that Weights the Spread Score and Spread Odds Equally

In [ ]:
spreads["win_score"] = 0.5 * spreads["spread_score"] + 0.5 * spreads["prob"]

: 

##### Win Probability Creation

In [ ]:
h2h = final[final['market'] == 'h2h'].copy()
h2h["win_probability"] = h2h["odds"].apply(converted_odds)

: 

##### Visualizing Sportsbook Bias Based on Totals

In [ ]:
totals = final[final["market"] == "totals"].copy()
totals["probability"] = totals["odds"].apply(converted_odds)

pivot = totals.pivot_table(index=["matchup", "sportsbook", "point"],
                           columns="team", values="probability").reset_index()

pivot["public_bias"] = pivot.get("Over", np.nan) - pivot.get("Under", np.nan)

bias = pivot.dropna(subset=["public_bias"])


: 

In [ ]:
value_df = h2h.merge(
    spreads[["team", "win_score"]], 
    on="team", 
    how="left"
)
# Value = Model probability (win_score) - Implied probability from sportsbook
value_df["value_score"] = value_df["win_score"] - value_df["win_probability"]

# Filter top 10 most underpriced teams
top_value_bets = value_df.sort_values(by="value_score", ascending=False).head(10)

# Optional: Only include odds where the payout is worthwhile
top_value_bets = top_value_bets[top_value_bets["odds"] > -200]

: 

# Visualizations

##### Predicting Winners Based off of Win Score 

In [ ]:

grouped = spreads.groupby(["matchup", "team"]).agg({"win_score": "mean"}).reset_index()

for matchup, metrics in grouped.groupby("matchup"):
    if len(metrics) != 2:
        continue
    teams = metrics["team"].values
    scores = metrics["win_score"].values
    winner = teams[np.argmax(scores)]

    colors = ["red", "red"]
    colors[np.argmax(scores)] = "green"

    x = np.arange(2)
    bars = plt.bar(x, scores, color=colors)
    for i, bar in enumerate(bars):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
                 f"{scores[i]:.2f}", ha="center")
    plt.xticks(x, teams)
    plt.title(f"{matchup}\nPredicted Winner: {winner}")
    plt.ylabel("Win Score")
    plt.tight_layout()
    plt.show()


: 

### Predicting Winners based on Win Probability

In [ ]:
if not h2h.empty:
    h2h_avg = h2h.groupby(["matchup", "team"])["win_probability"].mean().reset_index()
    for matchup, metrics in h2h_avg.groupby("matchup"):
        if len(metrics) != 2:
            continue
        teams = metrics["team"].values
        probs = metrics["win_probability"].values
        probs = probs/ probs.sum()

        x = np.arange(2)
        bars = plt.bar(x, probs, color="skyblue")
        for i, bar in enumerate(bars):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                     f"{probs[i]*100:.1f}%", ha="center")
        plt.xticks(x, teams)
        plt.title(f"{matchup} — H2H Win Probabilities")
        plt.ylabel("Probability")
        plt.tight_layout()
        plt.show()


: 

### Plotting Book Bias

In [ ]:
book_bias = bias.groupby("sportsbook")["public_bias"].mean().sort_values()

plt.figure(figsize=(8,5))
book_bias.plot(kind="barh", color="salmon", edgecolor="black")
plt.axvline(0, linestyle="--", color="black")
plt.title("Book Bias (Over vs Under)")
plt.xlabel("Bias")
plt.ylabel("Sportsbook")
plt.grid(axis="x", linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

: 